### Example 3: `argmax` -- nonworking solution.
As in `Example 2`, we
1. pass in an array to the inference.
2. Notice that the warmup example fixes the lengths of the array that the onnx model expects.
3. check out the netron visualization of the model.

In this example, we take an array `x`, and then take the `argmax` of the entries in `x`.
denote this index by `k`, then we return `m*k + c`.

**NOTE**: 
This solution does **not** work, even though Netron shows the exported ONNX model beautifully!

In [37]:
onnx_model = "example3.onnx"

In [38]:
import torch
import torch.nn as nn
import numpy as np

In [39]:
class SampleNet(nn.Module):
    def __init__(self):
        super(SampleNet, self).__init__()
        self.m = 2
        self.c = 1
    
    def forward(self, x):
        return self.m * torch.argmax(x) + self.c

In [40]:
model = SampleNet()
model.eval()

SampleNet()

In [41]:
# the warmup stage. 
x = torch.LongTensor([1, 2, 3, 4])
out = model(x)

In [42]:
out

tensor(7)

In [43]:
torch.onnx.export(
model,
x,
onnx_model,
opset_version=11,
do_constant_folding=True,
input_names=['input'],
output_names=['output'])

In [44]:
# uncomment to install netron.
#!pip install netron
import netron
netron.start(onnx_model, port=8082)


Stopping http://localhost:8082
Serving 'example3.onnx' at http://localhost:8082


### use the onnx model

In [27]:
import onnxruntime as ort
import numpy as np
sess = ort.InferenceSession(onnx_model)

NotImplemented: [ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for the node ArgMax_2:ArgMax(11)

In [28]:
sess.get_inputs()[0].name

NameError: name 'sess' is not defined

In [204]:
# check out the signature of sess.run - it has to have the output, then something like a feed_dict.

# either pass the feed dict directly.
# passing an array of any other size will result in an error.
outs = sess.run(['output'],
                       {
                         'input': np.array([11, 20, 22, 12], dtype=np.int64)  
                       })

print(outs)

[array(45, dtype=int64)]
